In [1657]:
#import libraries
import requests
import pandas as pd
import time
import isodate

In [1658]:
#keys
API_KEY = "API KEY"
#NepentheZ
CHANNEL_ID= "UCkaUtMpy3DU0L-q28pD62Ig"


In [1659]:
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count","duration", "caption","channel_statistics", "channel_views", "channel_subscribers", "video_caption", "comment_text"])



In [1660]:
def get_video_details(video_id):
    #statistics
    #collecting view, like, dislike, comment counts
        
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()
        
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    #dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']        
    
    return view_count, like_count, comment_count

In [1661]:
def get_video_details_2(video_id):
    #contentDetails
    #collecting duration
        
    url_video_stats_2 = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=contentDetails&key="+API_KEY
    response_video_stats_2 = requests.get(url_video_stats_2).json()
        
    duration_1 = response_video_stats_2['items'][0]['contentDetails']['duration']
    td = isodate.parse_duration(duration_1)
    duration = td.total_seconds() / 60
    caption = response_video_stats_2['items'][0]['contentDetails']['caption']
    
    return duration, caption

In [1662]:
def get_video_details_3(video_id):
    url_3 = "https://www.googleapis.com/youtube/v3/channels?part=statistics&id="+CHANNEL_ID+"&key="+API_KEY
    response_3 = requests.get(url_3).json()

    channel_statistics = response_3["items"][0]["statistics"]
    channel_views = channel_statistics["viewCount"]
    channel_subscribers = channel_statistics["subscriberCount"]

    #print(f"Channel views: {channel_views}")
    #print(f"Channel subscribers: {channel_subscribers}")
    
    return channel_statistics, channel_views, channel_subscribers
    

In [1663]:
def get_video_details_4(video_id):
    # Get video caption/description
    url_4 = "https://www.googleapis.com/youtube/v3/videos?part=snippet&id="+video_id+"&key="+API_KEY
    response_4 = requests.get(url_4).json()

    video_caption = response_4["items"][0]["snippet"]["description"]

    #print(f"Video caption: {video_caption}")

    # Get video comments
    url_4_1 = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId="+video_id+"&key="+API_KEY
    response_4_1 = requests.get(url_4_1).json()
    comment_text = ""
    for item in response_4_1["items"]:
        comment_text = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        #print(f"Comment: {comment_text}")
    
    return video_caption, comment_text

In [1664]:
def get_videos(df):
    #make API call

    pageToken= ""
    # Set publishedAfter and publishedBefore values for February 2022
    publishedAfter = "2022-12-01T00:00:00Z"
    publishedBefore = "2022-12-02T00:00:00Z"
    url="https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&publishedAfter="+publishedAfter+"&publishedBefore="+publishedBefore

    response = requests.get(url).json()
    time.sleep(1)
    #print("response:", response)
    print("pageToken:", pageToken)


    for video in response['items']:
        if video['id']['kind'] == "youtube#video":
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            video_title = str(video_title).replace("&amp;","")
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]
        
            view_count, like_count, comment_count = get_video_details(video_id)
            duration, caption = get_video_details_2(video_id)
            channel_statistics, channel_views, channel_subscribers = get_video_details_3(video_id)
            video_caption, comment_text = get_video_details_4(video_id)
            
            #channel_statistics, channel_views, channel_subscribers, video_caption, comment_text
        
            #save data in pandas df
            df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,'view_count':view_count,'like_count':like_count,'comment_count':comment_count, 'duration':duration, 'caption':caption, 'channel_statistics':channel_statistics, 'channel_views':channel_views, 'channel_subscribers':channel_subscribers, 'video_caption':video_caption, 'comment_text':comment_text},ignore_index=True)
            #df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,'view_count':view_count,'like_count':like_count,'comment_count':comment_count},ignore_index=True)
            
    return df

In [1665]:
#main

#build dataframe

#df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","dislike_count","comment_count"])
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count","duration", "caption", "channel_statistics", "channel_views", "channel_subscribers", "video_caption", "comment_text"])

df = get_videos(df)



pageToken: 


C:\temp\ipykernel_19644\573801937.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,'view_count':view_count,'like_count':like_count,'comment_count':comment_count, 'duration':duration, 'caption':caption, 'channel_statistics':channel_statistics, 'channel_views':channel_views, 'channel_subscribers':channel_subscribers, 'video_caption':video_caption, 'comment_text':comment_text},ignore_index=True)
C:\temp\ipykernel_19644\573801937.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'video_id':video_id,'video_title':video_title,'upload_date':upload_date,'view_count':view_count,'like_count':like_count,'comment_count':comment_count, 'duration':duration, 'caption':caption, 'channel_statistics':channel_statistics,

In [1666]:
df

,video_id,video_title,upload_date,view_count,like_count,comment_count,duration,caption,channel_statistics,channel_views,channel_subscribers,video_caption,comment_text
0,hLd8h__Vlyg,Dear EA Sports.....,2022-12-01,88904,4537,746,8.016667,false,"{'viewCount': '947625418', 'subscriberCount': ...",947625418,1960000,Dear EA Sports.....\n\nI got a little bit stre...,I’ve stopped playing after swaps
1,aV-dVjUyLRA,EA are REFUSING to give us content!,2022-12-01,43620,1359,258,3.95,false,"{'viewCount': '947625418', 'subscriberCount': ...",947625418,1960000,►► ROAD TO GLORY CHANNEL : https://www.youtube...,"Fifa getting the madden treatment, RIP"


In [1667]:

'''
Streamers
Channel ID
DJMaRiio
UCi7TVXyvrIwqeS9tfYD8UDA
MattHDGamer
UCCdv8O4SIcISvAW94XdZTEw
Manny
UC1crUC35Gk-U_-rv5sM4PtA
AA9Skillz
UCL7vy7MDOq9-tE-r6taQBlw
TBJZLPlays
UCVa7nsA_blpxzmfZWTPEsLQ
AJ3
UC9QqBypELbqUVlsg4zMrN0Q
TWOSYNC
UC8vXGfLggx4CAFsr2i0IoRA
Bateson87
UCJ47W_WzuzbHaONWB5a9i7w
BFordLancer
UCNWgHP_0lBoB8QqECjZTgoQ
ZwebackHD
UCgAkZwAyzJTguJtpSPYCejw
Masterbucks
UCihU6D2FXrJ4Mxji4dWNSeg
HecticTKS
UCstCY_VEmzH80BgCIo24cHA
'''


'\nStreamers\nChannel ID\nDJMaRiio\nUCi7TVXyvrIwqeS9tfYD8UDA\nMattHDGamer\nUCCdv8O4SIcISvAW94XdZTEw\nManny\nUC1crUC35Gk-U_-rv5sM4PtA\nAA9Skillz\nUCL7vy7MDOq9-tE-r6taQBlw\nTBJZLPlays\nUCVa7nsA_blpxzmfZWTPEsLQ\nAJ3\nUC9QqBypELbqUVlsg4zMrN0Q\nTWOSYNC\nUC8vXGfLggx4CAFsr2i0IoRA\nBateson87\nUCJ47W_WzuzbHaONWB5a9i7w\nBFordLancer\nUCNWgHP_0lBoB8QqECjZTgoQ\nZwebackHD\nUCgAkZwAyzJTguJtpSPYCejw\nMasterbucks\nUCihU6D2FXrJ4Mxji4dWNSeg\nHecticTKS\nUCstCY_VEmzH80BgCIo24cHA\n'

In [1668]:
df.to_excel(r'C:\Users\Pranav SA\Desktop\E-Sports\Assignment 1 game data\exportdata.xlsx', index=False)


In [1669]:
#os.system('start excel.exe "{}"'.format(r'C:\Users\Pranav SA\Desktop\E-Sports\Assignment 1 game data\exportdata.xlsx'))